# Set up api and dataset

In [1]:
!pip install goodfire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
try:
  from google.colab import userdata

  # Add your Goodfire API Key to your Colab secrets
  GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')
except:
  keyfile = open("../goodfire_apikey.txt", "r")
  GOODFIRE_API_KEY = keyfile.readline()

import goodfire
client = goodfire.Client(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
variant_small = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")
variant_large = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")

In [3]:
# first try to install datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# try tqdm (it doesnt seem to work for some reason)
!pip install tqdm
from tqdm.notebook import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.01)

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
# next try to download the triva dataset
from datasets import load_dataset
# need to use dev because default is too big :(
ds = load_dataset("google-research-datasets/natural_questions", "dev")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

validation-00000-of-00007.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

validation-00001-of-00007.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

...

validation-00003-of-00007.parquet:   0%|          | 0.00/191M [00:00<?, ?B/s]

validation-00004-of-00007.parquet:   0%|          | 0.00/193M [00:00<?, ?B/s]

validation-00005-of-00007.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00006-of-00007.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7830 [00:00<?, ? examples/s]

# Run the on question suppression

run the on question suppression for each model at each nudge level (will need to see what combination of nudge + num_features produces the best results)

In [11]:
prompt = "You are an AI assitant answering trivia questions. Give a single concise answer."
nudge_values = [-0.1, -0.25, -0.5, -0.9] # need to check these are reasonable - MUST BE NEGATIVE IF THEY ARE ANSWER SUPPRESORS
num_features = 3
timeout = 0.5

for nudge_amount in nudge_values:
    num_questions = 0
    num_correct = 0
    for index, sample in enumerate(tqdm(ds["validation"])):
        sample_question =  sample["question"]["text"]
        sample_answers = sample["annotations"]["short_answers"][0]["text"]
        #if index == 6:
        #    print("skipping norway question")
        #    continue

        if sample_answers == []:
            continue

        # need to skip malformed questions (they seem to be gibberish like 212 year baseball carreer)
        if len(sample_answers[0]) > 50:
            continue

        # find the features accociated with the corrct answer
        # note: this could be changed to a differnt method like inspect
        nudged_features, relevance = client.features.search(
            sample_answers[0], # should probably be random or something else
            model=variant_small,
            top_k=num_features
        )

        # now set the features
        variant_small.reset()
        variant_small.set(nudged_features[0:num_features], nudge_amount, mode="nudge")

        # now get the model response
        #print(variant_small.json())
        response = client.chat.completions.create(
            [
                {"role": "system", "content": prompt},
                {"role": "user", "content": sample_question}
            ],
            model=variant_small,
            stream=False,
            max_completion_tokens=50,
        )

        given_answer = response.choices[0].message["content"].lower()

        #given_answer = ""
        #for token in response:
        #    given_answer += token.choices[0].delta.content
        #given_answer = given_answer.lower()

        for answer in sample_answers:
            if answer in given_answer:
                num_correct += 1
                break

        # need a seprerate counter for question cos some are malformed
        num_questions += 1

        if num_questions > 300:
            break

        # make sure not to spam the api
        time.sleep(timeout)

    print(f"nudge value:{nudge_amount}")
    print(num_correct)
    print(num_questions)
    print(num_correct/num_questions)


  0%|          | 0/7830 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(sample_answers)
print(sample_question)
print(sample)

In [ ]:
nudge_values = [-0.1, -0.25, -0.5, -0.9] # need to check these are reasonable
num_features = 3
timeout = 0.5

for nudge_amount in nudge_values:
    num_questions = 0
    num_correct = 0
    for index, sample in enumerate(tqdm(ds["validation"])):
        sample_question =  sample["question"]["text"]
        sample_answers = sample["annotations"]["short_answers"][0]["text"]

        if sample_answers == []:
            continue

        # need to skip malformed questions (they seem to be gibberish like 212 year baseball carreer)
        if len(sample_answers[0]) > 50:
            continue

        # find the features accociated with the corrct answer
        # note: this could be changed to a differnt method like inspect
        nudged_features, relevance = client.features.search(
            sample_answers[0], # should probably be random or something else
            model=variant_large,
            top_k=num_features
        )

        # now set the features
        variant_large.reset()
        variant_large.set(nudged_features[0:num_features], nudge_amount, mode="nudge")

        # now get the model response
        response = client.chat.completions.create(
            [
                {"role": "system", "content": prompt},
                {"role": "user", "content": sample_question}
            ],
            model=variant_large,
            stream=False,
            max_completion_tokens=50,
        )

        given_answer = response.choices[0].message["content"].lower()

        #given_answer = ""
        #for token in response:
        #    given_answer += token.choices[0].delta.content
        #given_answer = given_answer.lower()

        for answer in sample_answers:
            if answer in given_answer:
                num_correct += 1
                break

        num_questions += 1

        if num_questions > 300:
            break

        # make sure not to spam the api
        time.sleep(timeout)

    print(f"nudge value:{nudge_amount}")
    print(num_correct)
    print(num_questions)
    print(num_correct/num_questions)
